<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/wandb-log/Generate_gifs_from_logged_images_on_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🎉 Generate gifs from logged images on Weights & Biases

This script accompanies a blog post which can be found here:
https://wandb.ai/_scott/gif-maker/reports/Create-gifs-from-images-logged-to-W-B---VmlldzoyMTI4ODEz/


# 🐝 Install Weights & Biases and login

In [ ]:
!pip install -q wandb tqdm

In [ ]:
import wandb
wandb.login()

# 🪄 Use `wandb.Api` to download the images and create the gifs

In [ ]:
from PIL import Image
from pathlib import Path
from tqdm import tqdm
import fnmatch
from IPython.display import Image as im, HTML
from random import sample
import cv2
import os
from base64 import b64encode

def check_if_multiple_logged_images(key, run):
    for log in run.scan_history(keys=[key]):
        if log[key]['_type'] == 'images/separated':
            return True
        elif log[key]['_type'] == 'image-file':
            return False
        else:
            return False

def get_number_of_images(key, run):
    for log in run.scan_history(keys=[key]):
        return log[key]['count']

def images_to_gif(image_fnames, fname):
    if not image_fnames: return 
    image_fnames.sort(key=lambda x: int(x.name.split('_')[-2])) #sort by step
    frames = [Image.open(image) for image in image_fnames]
    frame_one = frames[0]
    frame_one.save(f'{fname}.gif', format="GIF", append_images=frames,
               save_all=True, duration=DURATION, loop=0)
    w, h = frames[0].size
    cmd = f"ffmpeg -loglevel error -i {f'{fname}.gif'} -vcodec libx264 -crf 25 -pix_fmt yuv420p {f'{fname}.mp4'}"
    os.system(cmd)
    if not os.path.exists(f'{fname}.mp4'):
        print(f"Failed to create mp4 file.")

def make_gifs(key, run, extension):
    if check_if_multiple_logged_images(key, run):
        count = get_number_of_images(key, run)
        for i in range(count):
            image_fnames = list(Path('./media/images/').glob(f'{key}*{i}{extension}'))
            images_to_gif(image_fnames, f'{key}_{i}')
    else:
        image_fnames = list(Path('./media/images/').glob(f'{key}*{extension}'))
        images_to_gif(image_fnames, key)

def download_files(filenames_to_download, run):
    keys = set()
    print('Downloading Files')
    for file in tqdm(run.files()):
        if Path(file.name).is_file():
            continue
        if Path(file.name).name not in filenames_to_download:
          continue
        file.download()
    return keys

def sample_fnames(matching_fnames):
  length = len(matching_fnames)
  if length > NUM_IMAGES_PER_GIF:
    matching_fnames.sort(key=lambda x: int(x.split('_')[-2])) #sort by step 
    fnames = sample(matching_fnames, NUM_IMAGES_PER_GIF)
    return fnames
  else:
    return matching_fnames

def get_filenames_for_key(key, all_filenames, extension):
  if check_if_multiple_logged_images(key, run):
    count = get_number_of_images(key, run)
    filenames_for_key = []
    for i in range(count):
      matching_fnames = fnmatch.filter(all_filenames, f'{key}*{i}{extension}')
      filenames_for_key.extend(sample_fnames(matching_fnames))
    return filenames_for_key
  else: 
    matching_fnames = fnmatch.filter(all_filenames, f'{key}*{extension}')
    length = len(matching_fnames)
    return sample_fnames(matching_fnames)
  
def display_gif(path):
  print(f'Generated gif: {path}')
  display(im(data=open(path,'rb').read(), format='png'))
 
def display_mp4(path, video_width = 600):
  video_file = open(path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  print(f'Generated mp4: {path}')
  display(HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>"""))

def make_and_display_gifs(run):
  extension = ".png"
  all_filenames = [Path(file.name).name for file in run.files() if file.name.endswith(extension)]
  keys = set([Path(fname).stem.split('_')[0] for fname in all_filenames]) 
  for key in keys:
    download_files(get_filenames_for_key(key, all_filenames, extension), run)
  for key in keys:
    path = make_gifs(key, run, extension)
  for path in Path('/content/').glob('*.gif'):
    display_gif(path)
    display_mp4(path.name.replace('.gif', '.mp4', 1))

# 👩‍🎨 Generate your gif and mp4s from a run

Put your [run path](https://wandb.ai/_scott/gif-maker/reports/Create-gifs-from-images-logged-to-W-B---VmlldzoyMTI4NDQx#get-your-run-path-from-your-run-overview) in the form (or replace it within the code) to start creating your gif and mp4s. Right click them to save them once it's done, or you can navigate to `/content/` 👈.

In [ ]:
RUN_PATH = '_scott/omnimatte/1ngtqqwn' #@param {type:"string"}
NUM_IMAGES_PER_GIF = 50 #@param {type:"integer"}
DURATION =  200 #@param {type:"integer"}

api = wandb.Api()
run = api.run(RUN_PATH)
make_and_display_gifs(run)